In [1]:
import numpy as np
import pickle

In [28]:
#Get encoded_phis, time_stamps from Iris measurements.
#Format [0,1,2,3,4,5]
# 0 time unit
# 1 starting phase
# 2 final phase
# 3 output normalized powers for each measurement step (dim: 1x60) from one output
# 4 non-normalized powers for all the chip modes (dim: 6x60)
# 5 time where each measurement is taken
#[time unit (s), starting phase, final phase, output normalized powers
#for each measurement step (dim: 1x60) from one output (THIS IS THE ONE
#YOU ARE INTERESTED IN), non-normalized powers for all the chip modes
#(dim: 6x60), time where each measurement is taken]


with open(r".\chipdata\nina_measurementspi.pkl", "rb") as f:
    #pickle.dump(arr,f)
    chip_data = pickle.load(f)

print(chip_data)


[0.001, 0, 0.0, [0.000987250630307542, 0.000955549629868222, 0.0009506318916379413, 0.0009417874652824355, 0.000958354262020111, 0.0009712244429716892, 0.0009377340077542936, 0.0009465379602252362, 0.0009452694321075835, 0.000978599914310819, 0.0009711917885393281, 0.0009705847748985911, 0.0009341284208231836, 0.0009293020429875097, 0.0009502259528259325, 0.0009710876546352774, 0.0009528979948057052, 0.0009524160211004041, 0.000946141276642446, 0.0009403485055483329, 0.0009655418840153619, 0.0009849741903314802, 0.0009490741333731237, 0.0009526362637125724, 0.0009455291160433249, 0.0009562592147321633, 0.0009470838421513385, 0.0009734157247704235, 0.0009664737061018756, 0.000939985024046353, 0.0009378655167750431, 0.0009588084169250079, 0.0009395537792539132, 0.0009754296967659044, 0.0009439386622309268, 0.0009556724140770809, 0.0009448571588009396, 0.0009376776692087752, 0.0009437552801653238, 0.0009550790716527906, 0.000953121163691824, 0.0009460895686528582, 0.0009873129257400128, 0

In [30]:
print(len(chip_data[3]))

60


In [ ]:
def get_encoded_phis_time(chip_data):
    """ Use chip data to obtain inputs for get_detector_timebins function.
    #TODO: cut before and after output normalized powers
    for each measurement step are approximately constant. Then,
    stitch data together in one long np.array with ascending orders of starting and final phases.
    """
    
    phi_times = chip_data

    return phi_times

def get_detector_timebins(phi_times, equi_time: bool = True, bin_number: int = 10):
    """ Use data, the encoded phases and corresponding time stamps,
        from photonic chip experiments to obtain time bins for the detector.
        Additionally this can be used to adapt the digital training of the 
        chip to the curve of the continuous variable encoding 'sweep'.
        Arguments:
            phi_times: contains encoded phases with time stamps depending on detector time resolution.
            equi_time: either fit parameters of chip digitally such that equi-sized time bins
                       are used (if set to True). Otherwise, equi-distant bins in the encoded phases 
                       are used. TODO: determine experimentally which one works better (guess it will be equi-time.)
            bin_number: number of classical data points to be encoded continuously.
        Return: 
            time_stamps: time stamps for when detector should allocate measurements to next data point.
    """

    if equi_time == False:
        x_max = phi_times[:,1].max()
        x_min = phi_times[:,1].min()
        xbins = [x_min+m*(x_max-x_min)/bin_number for m in range(bin_number+1)]
        for m in range(bin_number+1):
            # find closest time stamp to xbins[m]
            # add time stamp to list
            time_stamps = []
    else: 
        # here we have equi-time bins, but then the encoded phase bins have different sizes
        # get time stamp zero for x_min
        t_min = phi_times[:,2].min()
        t_max = phi_times[:,2].max() # this should be the time point when max phi_enc becomes approximately constant
        time_stamps = [t_min+m*(t_max-t_min)/bin_number for m in range(bin_number+1)]
    
    return time_stamps